In [ ]:
!pip install pandas 
!pip install mysql-connector-python
!pip install sqlalchemy
!pip install python-dotenv


In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, types
import mysql.connector
from mysql.connector import Error


In [ ]:
def df_to_sql(df, table_name, connection_string):
    try:
        # Crear un motor de conexión a la base de datos
        engine = create_engine(connection_string)

        # Añadir una columna de índice como clave primaria si no existe
        if 'id' not in df.columns:
            df.insert(0, 'id', range(1, 1 + len(df)))

        # Obtener el tipo de datos de cada columna en el DataFrame
        dtype_dict = {col: types.VARCHAR(length=255) for col in df.columns}
        dtype_dict['id'] = types.INTEGER().with_variant(types.BIGINT(), 'sqlite')
        
        df.to_sql(name=table_name, con=engine, if_exists='replace', index=False, dtype=dtype_dict)
        # print(f"✅ Datos insertados en la tabla {table_name} exitosamente.")
        
    except Exception as e:
        # print(f"❌ Error al insertar datos en MySQL: {e}")
        pass


In [ ]:
try:
    df = pd.read_csv('../data/MunE19_c.csv', sep=';', encoding='ISO-8859-1')
    print(df.head())
except Exception as e:
    print(f"❌ Error al leer el archivo CSV: {e}")

In [ ]:
# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener las variables de entorno
host = os.getenv('HOST')
port = os.getenv('PORT')
database = os.getenv('DATABASE')
user = os.getenv('USER')
password = os.getenv('PASSWORD')

In [ ]:
try:
    # Conectar a MySQL
    conn = mysql.connector.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )

    if conn.is_connected():
        print('✅ Conexión establecida a MySQL')
        cursor = conn.cursor()

        # Añadir una columna de índice como clave primaria si no existe
        if 'id' not in df.columns:
            df.insert(0, 'id', range(1, 1 + len(df)))

        # Definir la cadena de conexión con los detalles correctos
        connection_string = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}'

        # Llamar a la función para convertir el DataFrame a SQL
        df_to_sql(df, 'elecc_europa_2019_municipios', connection_string)

    else:
        print('❌ No se pudo conectar a MySQL')

except Error as e:
    print(f"❌ Error al conectar a MySQL: {e}")

finally:
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print('🔒 Conexión cerrada.')
